**Importing needed libraries**

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

**Loading dataset**

In [141]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

**Information of data**

In [142]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [143]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [144]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


**Filling missing values of age**

In [145]:
train.fillna(train['Age'].mean(),inplace=True)
test.fillna(test['Age'].mean(),inplace=True)

**Removing Ticket and Cabin columns**

In [146]:
train=train.drop(['Ticket','Cabin'],axis=1)
test=test.drop(['Ticket','Cabin'],axis=1)

combine=[train,test]

**Converting Names into Titles**

In [147]:
for dataset in combine:
  dataset['Titles']=dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# pd.crosstab(train['Titles'],train['Sex'])

In [148]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Titles
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,Mr


**Replacing uncommon titles with Rare**

In [149]:
for dataset in combine:
  dataset['Titles']=dataset['Titles'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  
  dataset['Titles']=dataset['Titles'].replace('Mlle','Miss')
  dataset['Titles']=dataset['Titles'].replace('Ms','Miss')
  dataset['Titles']=dataset['Titles'].replace('Mme','Mrs')

**Converting the titles into ordinals**

In [150]:
title_map={'Mr':1,'Mrs':2,'Miss':3,'Master':4,'Rare':5}
for dataset in combine:
  dataset['Titles']=dataset['Titles'].map(title_map)
  dataset['Titles']=dataset['Titles'].fillna(0)

**Dropping the names and passengerId from dataset**

In [151]:
train=train.drop(['PassengerId','Name'],axis=1)
test=test.drop('Name',axis=1)

**Converting age into numeric**

In [152]:
train['Age']=train['Age'].astype(int)
test['Age']=test['Age'].astype(int)
combine=[train,test]

for dataset in combine:
  dataset['Sex']=dataset['Sex'].map({'female':1,'male':0}).astype(int)

**Create Age bands**

In [153]:
train['AgeBand'] = pd.cut(train['Age'], 5)
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 16.0]",0.550000
1,"(16.0, 32.0]",0.344762
2,"(32.0, 48.0]",0.403226
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


In [154]:
for dataset in combine:
  dataset.loc[dataset['Age']<=16,'Age']=0
  dataset.loc[(dataset['Age']>16)& (dataset['Age']<=32),'Age']=1
  dataset.loc[(dataset['Age']>32)& (dataset['Age']<=48),'Age']=2
  dataset.loc[(dataset['Age']>48)& (dataset['Age']<=64),'Age']=3
  dataset.loc[dataset['Age']>64,'Age']=4

In [155]:
train.drop('AgeBand',axis=1,inplace=True)
combine=[train,test]

In [156]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titles
0,0,3,0,1,1,0,7.2500,S,1
1,1,1,1,2,1,0,71.2833,C,2
2,1,3,1,1,0,0,7.9250,S,3
3,1,1,1,2,1,0,53.1000,S,2
4,0,3,0,2,0,0,8.0500,S,1


In [157]:
for dataset in combine:
  dataset['Family']=dataset['SibSp']+dataset['Parch']+1

**Converting Familysize into binary values alone or not**

In [158]:
for dataset in combine:
  dataset['Alone']=0
  dataset.loc[dataset['Family']==1,'Alone']=1

In [159]:
for dataset in combine:
  dataset.drop(['SibSp','Parch','Family'],axis=1,inplace=True)

In [160]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Titles,Alone
0,0,3,0,1,7.2500,S,1,0
1,1,1,1,2,71.2833,C,2,0
2,1,3,1,1,7.9250,S,3,1
3,1,1,1,2,53.1000,S,2,0
4,0,3,0,2,8.0500,S,1,1


**Finding most frequent in Embarked and filling nan with it**

In [161]:
freq_port = train.Embarked.dropna().mode()[0]
for dataset in combine:
  dataset['Embarked']=dataset['Embarked'].fillna(freq_port)

In [162]:
embark_map={'S':0,'C':1,'Q':2}
for dataset in combine:
  dataset['Embarked']=dataset['Embarked'].map(embark_map)

In [163]:
train['FareBand'] = pd.qcut(train['Fare'], 4)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [164]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [165]:
combine[0]

,Survived,Pclass,Sex,Age,Fare,Embarked,Titles,Alone,FareBand
0,0,3,0,1,0,0.0,1,0,"(-0.001, 7.91]"
1,1,1,1,2,3,1.0,2,0,"(31.0, 512.329]"
2,1,3,1,1,1,0.0,3,1,"(7.91, 14.454]"
3,1,1,1,2,3,0.0,2,0,"(31.0, 512.329]"
4,0,3,0,2,1,0.0,1,1,"(7.91, 14.454]"
5,0,3,0,1,1,2.0,1,1,"(7.91, 14.454]"
6,0,1,0,3,3,0.0,1,1,"(31.0, 512.329]"
7,0,3,0,0,2,0.0,4,0,"(14.454, 31.0]"
8,1,3,1,1,1,0.0,2,0,"(7.91, 14.454]"
9,1,2,1,0,2,1.0,2,0,"(14.454, 31.0]"


In [172]:
train['Embarked']=train['Embarked'].fillna(0)
train.drop(['FareBand'],axis=1,inplace=True)

In [173]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null int64
Fare        891 non-null int64
Embarked    891 non-null float64
Titles      891 non-null int64
Alone       891 non-null int64
dtypes: float64(1), int64(7)
memory usage: 55.8 KB


In [177]:
X=train[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Titles','Alone']]
y=train[['Survived']]

In [179]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [182]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

/home/sky/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/sky/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [183]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred,y_test)

array([[114,  25],
       [ 25,  59]])

In [185]:
from sklearn.tree import DecisionTreeClassifier
clas=DecisionTreeClassifier(criterion='entropy',random_state=0)

In [187]:
clas.fit(X_train,y_train)
y_dt_pred=clas.predict(X_test)

In [188]:
confusion_matrix(y_dt_pred,y_test)

array([[124,  25],
       [ 15,  59]])

In [189]:
from sklearn.neighbors import KNeighborsClassifier
knClas=KNeighborsClassifier(n_estimator=5)

TypeError: __init__() got an unexpected keyword argument 'n_estimator'